# Analyze database

## Imports

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Constants

In [2]:
df = pd.read_csv('data_k.csv', parse_dates=['Transaction Date'])

TRIP_TABLE_CATEGORIES = ['Muni Bus', 'Muni Metro', 'BART Entrance', 'Cable Car',
                         'Caltrain Entrance', 'Ferry Entrance', 'AC Transit', 'SamTrans']

COST_TABLE_CATEGORIES = ['Muni Bus', 'Muni Metro', 'BART Exit', 'Cable Car',
                         'Caltrain', 'Ferry', 'AC Transit', 'SamTrans']

COLOR_MAP = {'Muni Bus': '#BA0C2F', 'Muni Metro': '#FDB813', 'BART': '#0099CC',
             'Cable Car': 'brown', 'Caltrain': '#6C6C6C', 'AC Transit': '#00A55E',
             'Ferry': '#008080', 'SamTrans': '#D3D3D3'}

# Set the display options to show floats with 2 decimal places
pd.options.display.float_format = "{:.2f}".format

## Tables

### Trips by year

In [3]:
# Create pivot table by year and category
pivot_year = (df.pivot_table(index=df['Transaction Date'].dt.year,
                                      columns='Category',
                                      values='Transaction Date',
                                      aggfunc='count',
                                      fill_value=0
                                      ))

# Sort by date and rename index
pivot_year.sort_index(ascending=False, inplace=True)
pivot_year.index.name = 'Year'

# Reorder columns and remove 'Entrance' from column names
pivot_year = pivot_year.reindex(columns=TRIP_TABLE_CATEGORIES).fillna(0).astype(int)
pivot_year.columns = [c.replace(' Entrance', '') for c in pivot_year.columns]

# FIX FERRIES!!!

pivot_year

,Muni Bus,Muni Metro,BART,Cable Car,Caltrain,Ferry,AC Transit,SamTrans
Year,,,,,,,,
2024,33,1,5,0,28,0,2,0
2023,259,25,45,0,43,2,5,1
2022,232,126,48,0,1,9,11,1
2021,81,26,10,0,0,0,2,2


### Trips by month

In [4]:
# Create pivot table by month and category
pivot_month = (df.groupby([pd.Grouper(key='Transaction Date', freq='M'), 'Category'])
               .size()
               .unstack(fill_value=0)
               )

# Sort by date and rename index to month and year
pivot_month.sort_index(ascending=False, inplace=True)
pivot_month.index = pivot_month.index.strftime('%b %Y')
pivot_month.index.name = 'Month'

# Reorder columns and remove 'Entrance' from column names
pivot_month = pivot_month.reindex(columns=TRIP_TABLE_CATEGORIES).fillna(0).astype(int)
pivot_month.columns = [c.replace(' Entrance', '') for c in pivot_month.columns]


pivot_month

,Muni Bus,Muni Metro,BART,Cable Car,Caltrain,Ferry,AC Transit,SamTrans
Month,,,,,,,,
Jan 2024,33,1,5,0,28,0,2,0
Dec 2023,27,2,4,0,4,0,0,0
Nov 2023,19,1,4,0,15,0,0,0
Oct 2023,21,0,4,0,1,0,1,1
Sep 2023,22,1,5,0,23,2,0,0
Aug 2023,26,0,5,0,0,0,0,0
Jul 2023,15,0,3,0,0,0,0,0
Jun 2023,17,1,2,0,0,0,0,0
May 2023,36,0,1,0,0,0,1,0


### Cost by year

In [5]:
# Create pivot table by year and category
pivot_year_cost = (df.pivot_table(index=df['Transaction Date'].dt.year,
                                  columns='Category',
                                  values=['Debit', 'Credit'],
                                  aggfunc='sum',
                                  fill_value=0
                                  ))

# Calculate net values for BART, Caltrain, and Ferry
pivot_year_cost[('Debit', 'Caltrain')] = (pivot_year_cost[('Debit', 'Caltrain Entrance')] -
                                     pivot_year_cost[('Credit', 'Caltrain Exit')])
pivot_year_cost[('Debit', 'Ferry')] = (pivot_year_cost[('Debit', 'Ferry Entrance')] +
                                  pivot_year_cost[('Debit', 'Ferry Exit')] -
                                  pivot_year_cost[('Credit', 'Ferry Exit')])

# Drop credit columns
pivot_year_cost = pivot_year_cost['Debit']

# Sort by date and rename index
pivot_year_cost.sort_index(ascending=False, inplace=True)
pivot_year_cost.index.name = 'Year'

# Reorder columns and remove 'Entrance' from column names
pivot_year_cost = (pivot_year_cost.reindex(columns=COST_TABLE_CATEGORIES)
              .fillna(0))
pivot_year_cost.rename(columns={'BART Exit': 'BART'}, inplace=True)

pivot_year_cost

Category,Muni Bus,Muni Metro,BART,Cable Car,Caltrain,Ferry,AC Transit,SamTrans
Year,,,,,,,,
2024,62.00,2.00,20.60,0.00,208.40,0.00,4.00,0.00
2023,436.00,42.50,238.45,0.00,307.40,17.95,12.25,0.00
2022,358.00,226.50,244.95,0.00,5.45,32.75,25.00,2.05
2021,154.00,58.00,57.20,0.00,0.00,0.00,4.00,4.10


### Cost by month

In [6]:
# Create pivot table by month and category
pivot_month_cost = (df.groupby([pd.Grouper(key='Transaction Date', freq='M'), 'Category'])[['Debit', 'Credit']]
                .sum()
                .unstack(fill_value=0)
                )

# Calculate net values for BART, Caltrain, and Ferry
pivot_month_cost[('Debit', 'Caltrain')] = (pivot_month_cost[('Debit', 'Caltrain Entrance')] -
                                           pivot_month_cost[('Credit', 'Caltrain Exit')])
pivot_month_cost[('Debit', 'Ferry')] = (pivot_month_cost[('Debit', 'Ferry Entrance')] +
                                        pivot_month_cost[('Debit', 'Ferry Exit')] -
                                        pivot_month_cost[('Credit', 'Ferry Exit')])

# Drop credit columns
pivot_month_cost = pivot_month_cost['Debit']


# Sort by date and rename index to month and year
pivot_month_cost.sort_index(ascending=False, inplace=True)
pivot_month_cost.index = pivot_month_cost.index.strftime('%b %Y')
pivot_month_cost.index.name = 'Month'

# Reorder columns and remove 'Entrance' from column names
pivot_month_cost = pivot_month_cost.reindex(columns=COST_TABLE_CATEGORIES).fillna(0)
pivot_month_cost.rename(columns={'BART Exit': 'BART'}, inplace=True)

pivot_month_cost

Category,Muni Bus,Muni Metro,BART,Cable Car,Caltrain,Ferry,AC Transit,SamTrans
Month,,,,,,,,
Jan 2024,62.00,2.00,20.60,0.00,208.40,0.00,4.00,0.00
Dec 2023,44.00,2.50,34.30,0.00,30.80,0.00,0.00,0.00
Nov 2023,32.50,2.00,17.10,0.00,88.25,0.00,0.00,0.00
Oct 2023,42.00,0.00,17.15,0.00,0.00,0.00,6.00,0.00
Sep 2023,43.00,2.00,13.55,0.00,188.35,17.95,0.00,0.00
Aug 2023,32.50,0.00,18.50,0.00,0.00,0.00,0.00,0.00
Jul 2023,24.50,0.00,30.00,0.00,0.00,0.00,0.00,0.00
Jun 2023,30.00,0.00,4.05,0.00,0.00,0.00,0.00,0.00
May 2023,60.00,0.00,4.05,0.00,0.00,0.00,0.00,0.00


### Free transfers

In [7]:
free_xfers = ((df['Transaction Type'] == 'Single-tag fare payment') & (df['Debit'].isna())).sum()
print(f'{free_xfers} total free transfers')

243 total free transfers


## Charts

In [8]:
pivot_month.index = pd.to_datetime(pivot_month.index)
trip_chart = px.bar(pivot_month,
                    color_discrete_map=COLOR_MAP,
                    )

trip_chart.update_layout(
    title_text="K's monthly trips",
    xaxis_title='',
    yaxis_title='Number of trips',
    legend_title='',
    bargap=0.1)

trip_chart.update_traces(hovertemplate= '<b>%{x|%B %Y}</b>: %{y}')

/var/folders/04/3nd376_s4cg9q732ybychj4w0000gn/T/ipykernel_6033/4173318253.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pivot_month.index = pd.to_datetime(pivot_month.index)


In [9]:
pivot_month_cost.index = pd.to_datetime(pivot_month_cost.index)
cost_chart = px.bar(pivot_month_cost,
                    color_discrete_map=COLOR_MAP,
                    )

cost_chart.update_layout(
    title_text="K's monthly transit cost",
    xaxis_title='',
    yaxis_title='Cost in $',
    legend_title='',
    bargap=0.1)

cost_chart.update_traces(hovertemplate= '<b>%{x|%B %Y}</b>: $%{y}')

/var/folders/04/3nd376_s4cg9q732ybychj4w0000gn/T/ipykernel_6033/3959018159.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



## Scratch